In [12]:
import pandas as pd

us_df=pd.read_csv("USvideos.csv")
in_df=pd.read_csv("INvideos.csv")


us_df["country"] = "US"
in_df["country"] = "India"

df = pd.concat([us_df, in_df], ignore_index=True)

df["trending_date"] = pd.to_datetime(df["trending_date"], format="%y.%d.%m", errors="coerce")
df["publish_time"] = pd.to_datetime(df["publish_time"], errors="coerce")
df["publish_date"] = df["publish_time"].dt.date
df = df.drop(["video_id", "thumbnail_link", "description"], axis=1, errors="ignore")
df["tags"] = df["tags"].fillna('')


In [13]:
import sqlite3
conn = sqlite3.connect(":memory:")
df.to_sql("youtube_trending", conn, if_exists="replace", index=False)


78301

In [14]:
conn = sqlite3.connect(":memory:")  

df.to_sql("youtube_trending", conn, if_exists="replace", index=False)


78301

In [15]:
query1 = """
SELECT category_id, country, ROUND(AVG(views), 2) AS avg_views
FROM youtube_trending
GROUP BY category_id, country
ORDER BY avg_views DESC;
"""
avg_views_df = pd.read_sql_query(query1, conn)
display(avg_views_df)


,category_id,country,avg_views
0,10,US,6201003.12
1,20,India,4162462.21
2,1,US,3106250.20
3,30,India,3065001.25
4,29,US,2963884.07
5,10,India,2631115.73
6,20,US,2620830.63
7,1,India,2320355.55
8,24,US,2067883.20
9,17,US,2025969.03


In [16]:
query2 = """
SELECT trending_date, country, COUNT(*) AS video_count
FROM youtube_trending
GROUP BY trending_date, country
ORDER BY trending_date ASC;
"""
trend_count_df = pd.read_sql_query(query2, conn)
display(trend_count_df)


,trending_date,country,video_count
0,2017-11-14 00:00:00,India,200
1,2017-11-14 00:00:00,US,200
2,2017-11-15 00:00:00,India,200
3,2017-11-15 00:00:00,US,200
4,2017-11-16 00:00:00,India,200
...,...,...,...
405,2018-06-12 00:00:00,US,200
406,2018-06-13 00:00:00,India,170
407,2018-06-13 00:00:00,US,200
408,2018-06-14 00:00:00,India,172


In [17]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

df['title_sentiment'] = df['title'].apply(lambda x: sid.polarity_scores(str(x))['compound'])

df['title_sentiment_label'] = df['title_sentiment'].apply(
    lambda x: 'Positive' if x > 0.05 else 'Negative' if x < -0.05 else 'Neutral'
)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\AK\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [18]:
print("title_sentiment_label" in df.columns) 

True


In [20]:
df.to_sql("youtube_trending", conn, if_exists="replace", index=False)


78301

In [21]:
query3 = """
SELECT country, title_sentiment_label, COUNT(*) AS sentiment_count
FROM youtube_trending
GROUP BY country, title_sentiment_label;
"""
sentiment_df = pd.read_sql_query(query3, conn)
display(sentiment_df)


,country,title_sentiment_label,sentiment_count
0,India,Negative,4436
1,India,Neutral,25324
2,India,Positive,7592
3,US,Negative,7748
4,US,Neutral,22680
5,US,Positive,10521


In [23]:
query4 = """
SELECT category_id, COUNT(*) AS total_videos
FROM youtube_trending
GROUP BY category_id
ORDER BY total_videos DESC
LIMIT 10;
"""
top_categories_df = pd.read_sql_query(query4, conn)
display(top_categories_df)


,category_id,total_videos
0,24,26676
1,10,10330
2,25,7728
3,23,6886
4,22,5834
5,26,4991
6,1,4003
7,28,2953
8,17,2905
9,27,2883


In [11]:
conn_file = sqlite3.connect("youtube_trending.db")
df.to_sql("youtube_trending", conn_file, if_exists="replace", index=False)
conn_file.close()
